In [1]:
import requests, re, json
from bs4 import BeautifulSoup
from tqdm import tqdm

In [3]:

class CoronaVirusSipder(object):
    
    def __init__(self):
        self.home_url = "https://ncov.dxy.cn/ncovh5/view/pneumonia"
        
    def get_content_from_url(self, url):
        response = requests.get(url)
        return response.content.decode()
    
    def parse_home_page(self, homepage):
        soup = BeautifulSoup(homepage, 'lxml')
        script = soup.find(id="getListByCountryTypeService2true")
        text = script.string
        
        
        json_str = re.findall(r'\[.+\]',text)[0]
        
        data = json.loads(json_str)
        return data
    
    def save(self, data, path):
        with open(path,"w",encoding='utf-8') as fp:
            json.dump(data, fp, ensure_ascii=False)
            
    def crawl_last_day_corona_virus(self):
        #1.获取首页内容
        homepage = self.get_content_from_url(self.home_url)
        #2.解析数据
        last_day_corona_virus = self.parse_home_page(homepage)
        #3.保存数据
        self.save(last_day_corona_virus, "data\\last_day_corona_virus.json")
        
    def crawl_corona_virus(self):
        """
        采集从1月23号以来各国疫情数据
        """
        #1.加载各国疫情数据
        with open("data\\last_day_corona_virus.json",encoding="utf-8") as fp:
            last_day_corona_virus = json.load(fp)
        corona_virus = []
        #2.便利各国疫情数据statisticsData，获取统计的URL
        for country in tqdm(last_day_corona_virus, "采集1月23日以来各国信息"):
             #3.发送请求，获取各国从1月23日至今的json数据
            statistic_data_url = country['statisticsData']
            statistic_data_json_str = self.get_content_from_url(statistic_data_url)
            #4.把json数据转换为Python数据，添加到列表中
            statistic_data = json.loads(statistic_data_json_str)['data']
            for one_day in statistic_data:
                one_day['provinceName'] = country['provinceName']
                one_day['countryShortCode'] = country['countryShortCode']
            corona_virus.extend(statistic_data)
        #5.保存列表数据到json文件
        self.save(corona_virus, "data\\corona_virus.json")
        
        
    def run(self):
        #self.crawl_last_day_corona_virus()
        self.crawl_corona_virus()
        
        
if __name__ == '__main__':
    spider = CoronaVirusSipder()
    spider.run()

采集1月23日以来各国信息:   0%|          | 0/215 [00:01<?, ?it/s]


TypeError: 'str' object does not support item assignment